In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ['HF_HOME'] = '/shared/data3/pk36/.cache'
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
!export HF_HOME=/shared/data3/pk36/.cache
os.environ['CUDA_VISIBLE_DEVICES'] = "4,5"

In [3]:
from debate import DebateNode
from paper_details import Paper
from persona import PaperAuthor
from moderator import Moderator
import argparse
from typing import List
from vllm import LLM
import os
import json
from data_pairer import parse_papers
import numpy as np

In [6]:
class Args():
    def __init__(self):
        self.log_dir = "logs"
        self.focus_paper = "https://arxiv.org/pdf/1706.03762"
        self.cited_paper = "https://arxiv.org/pdf/1810.04805"
        self.topic = "language model architectures"

args = Args()

In [7]:
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

# if not os.path.exists("data.json"):
parse_papers(args.focus_paper, args.cited_paper)
with open('data.json', 'r') as file:
    data = json.load(file)

In [8]:
model_server = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",tensor_parallel_size=2, gpu_memory_utilization=0.5, max_num_seqs=100) #,enable_prefix_caching=True)

INFO 11-20 18:03:06 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-20 18:03:06 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-20 18:03:06 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-20 18:03:06 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 11-20 18:03:16 model_runner.py:1067] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=3521382) INFO 11-20 18:03:17 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-20 18:03:22 distributed_gpu_executor.py:57] # GPU blocks: 15590, # CPU blocks: 4096
INFO 11-20 18:03:22 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 1.90x
INFO 11-20 18:03:24 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-20 18:03:24 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
(VllmWorkerProcess pid=3521382) INFO 11-20 18:03:24 model_runner.py:1395] Capturing the mo

In [9]:
item = data[0]
f_pap = item['focus']
c_pap = item['cited']

focus_paper = PaperAuthor(
    model = model_server,
    paper = Paper(f_pap),
    focus=True,
    id=0
)

cited_paper = PaperAuthor(
    model = model_server,
    paper = Paper(c_pap),
    focus=False,
    id=1
)

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

100%|██████████| 2/2 [04:48<00:00, 144.29s/it]


In [10]:
focus_paper.paper.retrieve_top_k("language model architectures", k=5)

100%|██████████| 1/1 [00:00<00:00,  3.05it/s]


[('We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. Experiments on two machine translation tasks show these models to be superior in quality while being more parallelizable and requiring significantly less time to train',
  0.73787904),
 ('Introduction Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5]. Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15]',
  0.7326936),
 ('We show that the Transformer generalizes well to other tasks by applying it successfully to English constituency parsing both with large and limited training data. *Equal contribution',
  0.7009049),
 ('Thi

In [11]:
focus_paper.paper.retrieve_top_k("multi headed attention", k=5)

100%|██████████| 1/1 [00:00<00:00,  3.51it/s]


[('With a single attention head, averaging inhibits this. MultiHead(Q, K, V ) = Concat(head1, ..., headh)W O where headi = Attention(QW Q i , KW Ki , V W Vi ) Where the projections are parameter matrices W Q and W O Rhdvxdmodel',
  0.8372327),
 ('We inspect attention distributions from our models and present and discuss examples in the appendix. Not only do individual attention heads clearly learn to perform different tasks, many appear to exhibit behavior related to the syntactic and semantic structure of the sentences',
  0.7794119),
 ('These are concatenated and once again projected, resulting in the final values, as depicted in Figure 2. Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions',
  0.77801174),
 ('The first is a multi-head self-attention mechanism, and the second is a simple, positionwise fully connected feed-forward network. We employ a residual connection [11] around each of the two sub-l

In [ ]:
moderator = Moderator(model_server)

paper_authors = [focus_paper, cited_paper]
leaf_node_label = args.topic

if args.log_dir != "":
    with open(os.path.join(args.log_dir, 'self_deliberation.txt'), 'w') as f:
        f.write(f'Topic: {args.topic}\n\n')

# each node has a topic
root_node = DebateNode(leaf_node_label)
subtrees = root_node.conduct_self_deliberation(leaf_node_label, paper_authors, log=args.log_dir) # k new, finer topics to discuss

In [24]:
evidence, scores = paper_authors[1].gather_evidence(args.topic, k=5, return_scores=True)

100%|██████████| 1/1 [00:00<00:00,  3.00it/s]


In [25]:
evidence

['They extract context-sensitive features from a left-to-right and a right-to-left language model. The contextual representation of each token is the concatenation of the left-to-right and right-to-left representations',
 'The major limitation is that standard language models are unidirectional, and this limits the choice of architectures that can be used during pre-training. For example, in OpenAI GPT, the authors use a left-toright architecture, where every token can only attend to previous tokens in the self-attention layers of the Transformer (Vaswani et al., 2017)',
 '4We note that in the literature the bidirectional TransBERTBERTE[CLS]E1 E[SEP]...ENE1\'...EM\'CT1T[SEP]...TNT1\'...TM\'[CLS]Tok 1 [SEP]...Tok NTok 1...TokMQuestionParagraphStart/End SpanBERTE[CLS]E1 E[SEP]...ENE1\'...EM\'CT1T[SEP]...TNT1\'...TM\'[CLS]Tok 1 [SEP]...Tok NTok 1...TokMMasked Sentence AMasked Sentence BPre-trainingFine-TuningNSPMask LMMask LMUnlabeled Sentence A and B Pair SQuADQuestion Answer PairNERMNLI

In [34]:
topic = args.topic
log = None
focus_paper = None
arguments = {}
children = []
evidences = {}

focus_paper = None
for paper_author in paper_authors:
    # gather evidence
    evidence, scores = paper_author.gather_evidence(topic, return_scores=True)

    if paper_author.id not in evidences.keys(): evidences[paper_author.id] = []
    evidences[paper_author.id].append(evidence)

    # develop k arguments
    if paper_author.id not in arguments.keys(): arguments[paper_author.id] = []
    author_args = paper_author.generate_arguments(topic, evidence,k=2)
    arguments[paper_author.id].append(author_args)

    # check if paper is the focus
    if paper_author.focus:
        focus_paper = paper_author

    # logging
    if log is not None:
        with open(os.path.join(log, 'self_deliberation.txt'), 'a') as f:
            f.write(f'Topic: {topic}\n\n')
            f.write(f'Gather Evidence:\n\n')
            temp = "\n".join([f'{s} - {e}' for s, e in zip(scores, evidence)])
            f.write(f'{paper_author.focus} paper:\n{temp}\n\n')

            f.write(f'Develop Arguments:\n\n')
            f.write(f'{paper_author.focus} paper:\n{author_args}\n\n')

# preemption
for i in range(len(paper_authors)):
    other_arguments = [arguments[paper_authors[j].id] for j in range(len(paper_authors)) if j != i]
    other_evidence = [evidences[paper_authors[j].id] for j in range(len(paper_authors)) if j != i]

    preemption = paper_authors[i].preempt_arguments(other_arguments, other_evidence)
    
    # logging
    if log is not None:
        with open(os.path.join(log, 'self_deliberation.txt'), 'a') as f:
            f.write(f'Preemption:\n\n')
            f.write(f'{paper_author.focus} paper:\n{preemption}\n\n')

    evidences[paper_authors[i].id].append(preemption)         

# for child_topic in arguments[focus_paper.id]:
#     children.append(DebateNode(child_topic, parent=self))

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.11s/it, est. speed input: 99.07 toks/s, output: 56.63 toks/s]


{"argument_list": [{"title": "RNNs are outperform by newer language model architectures", "description": "Newer architectures such as attention-based models, have outperformed traditional RNN-based models in sequence modeling and transduction problems such as machine translation and language modeling.", "evidence": ["Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks in particular, have been firmly established as state of the art approaches in sequence modeling and transduction problems such as language modeling and machine translation [35, 2, 5].", "Numerous efforts have since continued to push the boundaries of recurrent language models and encoder-decoder architectures [38, 24, 15].", "Recurrent models typically factor computation along the symbol positions of the input and output sequences.", "On both WMT 2014 English-to-German and WMT 2014 English-to-French translation tasks, we achieve a new state of the art.", "In the former task our b

Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.08s/it, est. speed input: 106.74 toks/s, output: 55.48 toks/s]


{"argument_list": [{"title": "Bidirectional Transformer architectures outperform unidirectional architectures on a variety of linguistic tasks.", "description": "Extensive studies have shown that the pre-training objectives, Masked Language Modeling (MLM) and Next Sentence Prediction (NSP), lead to the development of powerful bidirectional transformer architectures, such as BERT, which achieve state-of-the-art results on a variety of NLP tasks, including question answering, natural language inference, and sentiment analysis. In addition, the ability of BERT to handle a variety of downstream tasks is a result of its input representation, which can unambiguously represent both a single sentence and a pair of sentences in one token sequence. Furthermore, studies have shown that fine-tuning pre-trained Bidirectional architectures with downstream specific objectives can unlock even state-of-the-art results, demonstrating the superiority of Bidirectional transformer architectures. \"Evidence

In [44]:
evidences[0][1]

{'Does my paper also include or address [{\'argument_list\': [{\'title\': \'Bidirectional Transformer architectures outperform unidirectional architectures on a variety of linguistic tasks.\', \'description\': \'Extensive studies have shown that the pre-training objectives, Masked Language Modeling (MLM) and Next Sentence Prediction (NSP), lead to the development of powerful bidirectional transformer architectures, such as BERT, which achieve state-of-the-art results on a variety of NLP tasks, including question answering, natural language inference, and sentiment analysis. In addition, the ability of BERT to handle a variety of downstream tasks is a result of its input representation, which can unambiguously represent both a single sentence and a pair of sentences in one token sequence. Furthermore, studies have shown that fine-tuning pre-trained Bidirectional architectures with downstream specific objectives can unlock even state-of-the-art results, demonstrating the superiority of B

: 

In [ ]:
conversation_history = []

queue_of_rounds: List[DebateNode] = []
queue_of_rounds.extend(subtrees)


while len(queue_of_rounds) > 0:
    round = queue_of_rounds.pop(0)
    conversation, new_focus_arg, new_cited_arg = round.conduct_debate(focus_paper, cited_paper)
    conversation_history.extend(conversation)
    if moderator.is_expand(round.arguments, [new_focus_arg, new_cited_arg]):
        new_subtrees = round.conduct_self_deliberation(round.round_topic, paper_authors)
        queue_of_rounds.extend(new_subtrees)

with open('conversation_history.txt', 'w+') as f:
    f.write('\n'.join(conversation_history))